In [2]:
from tkinter import *
import pyautogui
import cv2
import numpy as np
from PIL import Image
import pytesseract
import subprocess
import requests
import urllib.parse
import time

time.sleep(4)


def getTranslation(text):
 txt = text.replace("\n","")
 URL ="https://translate.googleapis.com/translate_a/single?client=gtx&sl=en&tl=bn&dt=t&q="+urllib.parse.quote(txt);
 r = requests.get(url = URL)
 data = r.json()
 showText(data)
 return data;

def showText(text):
 root = Tk()
 AdviseBox = Text(root, width=50, height=20)
 AdviseBox.pack(side=RIGHT, expand=YES, padx=5, pady=5)
 AdviseBox.insert(INSERT, text)
 root.mainloop()
def openNotepad(text):
 subprocess.Popen(["notepad","filename.txt"])
def getText(c_image):
 im = Image.open("s/screen.png")
 pytesseract.pytesseract.tesseract_cmd = r'D:/PortableApps/ocrlib/tesseract.exe'
 text = pytesseract.image_to_string(c_image, lang = 'eng')
 return text

def getScreenShot():
 screenshot = pyautogui.screenshot()
 screenshot.save("s/screen.png")

getScreenShot();
image = cv2.imread('s/screen.png')
oriImage = image.copy()
cv2.imshow("image", image)



cropping = False
x_start, y_start, x_end, y_end = 0, 0, 0, 0
def mouse_crop(event, x, y, flags, param):
    # grab references to the global variables
    global x_start, y_start, x_end, y_end, cropping

    # if the left mouse button was DOWN, start RECORDING
    # (x, y) coordinates and indicate that cropping is being
    if event == cv2.EVENT_LBUTTONDOWN:
        x_start, y_start, x_end, y_end = x, y, x, y
        cropping = True

    # Mouse is Moving
    elif event == cv2.EVENT_MOUSEMOVE:
        if cropping == True:
            x_end, y_end = x, y

    # if the left mouse button was released
    elif event == cv2.EVENT_LBUTTONUP:
        # record the ending (x, y) coordinates
        x_end, y_end = x, y
        cropping = False # cropping is finished

        refPoint = [(x_start, y_start), (x_end, y_end)]

        if len(refPoint) == 2: #when two points were found
            roi = oriImage[refPoint[0][1]:refPoint[1][1], refPoint[0][0]:refPoint[1][0]]
            #cv2.imshow("Cropped", roi)
            text = getText(roi)
            showText(text)
            getTranslation(text)


cv2.namedWindow("image")
cv2.setMouseCallback("image", mouse_crop)


while True:
    i = image.copy()
    k = cv2.waitKey(200)
    if not cropping:
        cv2.imshow("image", image)

    elif cropping:
        cv2.rectangle(i, (x_start, y_start), (x_end, y_end), (255, 0, 0), 2)
        cv2.imshow("image", i)
    if k in [27, 1048603]: # ESC key to abort, close window
        cv2.destroyAllWindows()
        break
    

